# Homework and bake-off: Relation extraction using distant supervision

In [ ]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Fall 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [ ]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [ ]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [ ]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [ ]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [ ]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [ ]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [ ]:
splits

{'all': Corpus with 331,696 examples; KB with 45,884 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples}

## Baselines

### Hand-build feature functions

In [ ]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [ ]:
featurizers = [simple_bag_of_words_featurizer]

In [ ]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [ ]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.856      0.368      0.676        340       5716
author                    0.819      0.542      0.743        509       5885
capital                   0.500      0.221      0.399         95       5471
contains                  0.787      0.603      0.742       3904       9280
film_performance          0.800      0.578      0.743        766       6142
founders                  0.813      0.389      0.668        380       5756
genre                     0.659      0.171      0.419        170       5546
has_sibling               0.855      0.248      0.575        499       5875
has_spouse                0.894      0.328      0.665        594       5970
is_a                      0.708      0.219      0.490        497       5873
nationality               0.654      0.176      0.424        301       5677
parents     

Studying model weights might yield insights:

In [ ]:
rel_ext.examine_model_weights(baseline_results)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Highest and lowest feature weights for relation adjoins:

     2.520 Córdoba
     2.438 Valais
     2.374 Taluks
     ..... .....
    -1.171 other
    -1.200 Iraq
    -2.298 Earth

Highest and lowest feature weights for relation author:

     2.972 author
     2.468 poem
     2.430 book
     ..... .....
    -2.362 directed
    -2.707 Alice
    -7.012 1865

Highest and lowest feature weights for relation capital:

     2.772 capital
     1.702 especially
     1.641 km
     ..... .....
    -1.164 and
    -1.189 North
    -1.313 ’

Highest and lowest feature weights for relation contains:

     2.553 districts
     2.471 bordered
     2.174 county
     ..... .....
    -2.296 who
    -2.519 film
    -2.636 Brooklyn

Highest and lowest feature weights for relation film_performance:

     4.267 starring
     3.604 alongside
     3.410 co-starring
     ..... .....
    -1.822 novels
    -2.056 spy
    -4.001 double

Highest and lowest feature weights for relation founders:

     4.121 founded


### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [ ]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [ ]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [ ]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)


def glove_middle_featurizer_mean(kbt, corpus, np_func=np.mean):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)


def glove_middle_featurizer_max(kbt, corpus, np_func=np.max):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [ ]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.867      0.459      0.736        340       5716
author                    0.887      0.417      0.724        509       5885
capital                   0.629      0.232      0.468         95       5471
contains                  0.653      0.414      0.585       3904       9280
film_performance          0.837      0.336      0.644        766       6142
founders                  0.853      0.245      0.570        380       5756
genre                     0.611      0.065      0.227        170       5546
has_sibling               0.855      0.261      0.587        499       5875
has_spouse                0.881      0.350      0.676        594       5970
is_a                      0.704      0.139      0.388        497       5873
nationality               0.620      0.223      0.457        301       5677
parents     

In [ ]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer_max],
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.763      0.397      0.644        340       5716
author                    0.885      0.379      0.699        509       5885
capital                   0.680      0.179      0.436         95       5471
contains                  0.583      0.531      0.572       3904       9280
film_performance          0.822      0.313      0.620        766       6142
founders                  0.920      0.182      0.507        380       5756
genre                     0.769      0.059      0.225        170       5546
has_sibling               0.874      0.152      0.449        499       5875
has_spouse                0.885      0.298      0.635        594       5970
is_a                      0.663      0.119      0.346        497       5873
nationality               0.696      0.183      0.446        301       5677
parents     

In [ ]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer_mean],
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.341      0.041      0.139        340       5716
author                    0.865      0.354      0.671        509       5885
capital                   0.467      0.074      0.226         95       5471
contains                  0.581      0.518      0.567       3904       9280
film_performance          0.738      0.265      0.544        766       6142
founders                  0.656      0.105      0.321        380       5756
genre                     0.667      0.059      0.217        170       5546
has_sibling               0.796      0.086      0.301        499       5875
has_spouse                0.833      0.202      0.513        594       5970
is_a                      0.706      0.048      0.190        497       5873
nationality               0.727      0.133      0.384        301       5677
parents     

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [ ]:
def run_svm_model_factory():
    from sklearn.svm import SVC
    ##### YOUR CODE HERE
    svm_exp = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        model_factory = lambda: SVC(kernel='linear'),
        featurizers=featurizers,  # [glove_middle_featurizer],
        # vectorize=False, # Crucial for this featurizer!
        verbose=True)
    
    return svm_exp


def run_rbf_svm_model_factory():
    from sklearn.svm import SVC
    ##### YOUR CODE HERE
    svm_exp = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        model_factory = lambda: SVC(),
        featurizers=featurizers,  # [glove_middle_featurizer],
        # vectorize=False, # Crucial for this featurizer!
        verbose=True)
    
    return svm_exp

In [ ]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.759      0.324      0.598        340       5716
author                    0.762      0.603      0.724        509       5885
capital                   0.591      0.274      0.480         95       5471
contains                  0.777      0.609      0.736       3904       9280
film_performance          0.755      0.629      0.726        766       6142
founders                  0.721      0.421      0.631        380       5756
genre                     0.610      0.294      0.502        170       5546
has_sibling               0.813      0.244      0.555        499       5875
has_spouse                0.853      0.342      0.657        594       5970
is_a                      0.561      0.286      0.471        497       5873
nationality               0.495      0.179      0.366        301       5677
parents     

In [ ]:
# RBF does worse than linear kernel!
_ = run_rbf_svm_model_factory()

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.912      0.303      0.650        340       5716
author                    0.852      0.409      0.700        509       5885
capital                   0.750      0.126      0.377         95       5471
contains                  0.785      0.546      0.721       3904       9280
film_performance          0.826      0.516      0.737        766       6142
founders                  0.972      0.276      0.647        380       5756
genre                     0.647      0.065      0.231        170       5546
has_sibling               0.869      0.186      0.502        499       5875
has_spouse                0.914      0.251      0.598        594       5970
is_a                      0.832      0.159      0.450        497       5873
nationality               0.786      0.073      0.266        301       5677
parents     

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [ ]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter):
    # Append these to the end of the keys you add/access in
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            key = word + subject_object_suffix
            feature_counter[key] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            key = word + object_subject_suffix
            feature_counter[key] += 1
    return feature_counter



# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
exp_dir_bow = rel_ext.experiment(
    splits,
    featurizers=[directional_bag_of_words_featurizer])

features = exp_dir_bow['vectorizer'].get_feature_names_out()  # get the list of all features
print(f'There are {len(features)} directional bag-of-words features')

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.855      0.400      0.697        340       5716
author                    0.841      0.591      0.775        509       5885
capital                   0.634      0.274      0.502         95       5471
contains                  0.809      0.670      0.777       3904       9280
film_performance          0.837      0.655      0.793        766       6142
founders                  0.836      0.416      0.695        380       5756
genre                     0.712      0.276      0.541        170       5546
has_sibling               0.855      0.248      0.575        499       5875
has_spouse                0.902      0.357      0.691        594       5970
is_a                      0.753      0.245      0.533        497       5873
nationality               0.639      0.229      0.471        301       5677
parents     

In [ ]:
# check separation between symmetric releation ('has_sibling', 'adjoins') and other relations


def partial_directional_bag_of_words_featurizer(kbt, corpus, feature_counter):
    """
    Differentiate betweeen symmetric relations ('has_sibling', 'adjoins') and asymmetric relations

    Parameters
    ==========
    kbt: KBTriple
        a KBTriple from the KB 
    
    corpus: Corpus

    feature_counter: Counter
        An empty Counter to count occurrences for each entity pair

    Returns
    =======
    Counter
        a Counter that takes into account directions only on non-symmetric relations
    """
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            key = word + subject_object_suffix
            if kbt.rel == 'has_sibling' or  kbt.rel == 'adjoins':
                key = word
            feature_counter[key] += 1 
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            key = word + object_subject_suffix
            if kbt.rel == 'has_sibling' or  kbt.rel == 'adjoins':
                key = word
            feature_counter[key] += 1
    return feature_counter

In [ ]:
exp_dir_bow = rel_ext.experiment(
    splits,
    featurizers=[partial_directional_bag_of_words_featurizer])

features = exp_dir_bow['vectorizer'].get_feature_names_out()  # get the list of all features
print(f'There are {len(features)} directional bag-of-words features')

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.837      0.362      0.663        340       5716
author                    0.862      0.599      0.792        509       5885
capital                   0.679      0.200      0.459         95       5471
contains                  0.819      0.670      0.784       3904       9280
film_performance          0.826      0.644      0.782        766       6142
founders                  0.854      0.416      0.705        380       5756
genre                     0.631      0.241      0.477        170       5546
has_sibling               0.840      0.242      0.563        499       5875
has_spouse                0.844      0.345      0.655        594       5970
is_a                      0.699      0.229      0.496        497       5873
nationality               0.629      0.219      0.458        301       5677
parents     

In [ ]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [ ]:
s = 'The/DT dog/N napped/V'
splitted = s.split()
[tok_pos.replace('/', ' ') for tok_pos in splitted]

['The DT', 'dog N', 'napped V']

In [ ]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for bigram_tag in get_tag_bigrams(ex.middle_POS):
            feature_counter[bigram_tag] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for bigram_tag in get_tag_bigrams(ex.middle_POS):
            feature_counter[bigram_tag] += 1

    return feature_counter


def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"
    ##### YOUR CODE HERE
    tags = get_tags(s)
    tags = [start_symbol] + tags + [end_symbol]  # a start/end symbol for bigram tagging
    tag_bigrams = [f'{tags[i-1]} {tags[i]}' for i in range(1, len(tags))]
    return tag_bigrams




def get_tags(s):
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE

_ = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[middle_bigram_pos_tag_featurizer])


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.800      0.341      0.630        340       5716
author                    0.739      0.316      0.583        509       5885
capital                   0.405      0.158      0.309         95       5471
contains                  0.760      0.601      0.722       3904       9280
film_performance          0.720      0.443      0.640        766       6142
founders                  0.579      0.174      0.395        380       5756
genre                     0.484      0.176      0.359        170       5546
has_sibling               0.646      0.164      0.407        499       5875
has_spouse                0.745      0.271      0.552        594       5970
is_a                      0.577      0.159      0.378        497       5873
nationality               0.381      0.080      0.217        301       5677
parents     

In [ ]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
dog = wn.synsets('dog', pos='n')
dog

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]

In [ ]:
str(dog[0])

"Synset('dog.n.01')"

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')


def synset_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for synset in get_synsets(ex.middle_POS):
            feature_counter[synset] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for synset in get_synsets(ex.middle_POS):
            feature_counter[synset] += 1

    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets
    associated with elements of `s`.
    """
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]
    wt = [ (w, convert_tag(t))  for (w, t) in wt]  # convert tags s.t. it can be used by WordNet
    synsets = [str(synset) for (w, t) in wt for synset in wn.synsets(w, pos=t)]
    return synsets
    ##### YOUR CODE HERE




def convert_tag(t):
    """Converts tags so that they can be used by WordNet:

    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE

_ = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[synset_featurizer])


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.824      0.318      0.625        340       5716
author                    0.787      0.444      0.682        509       5885
capital                   0.488      0.221      0.393         95       5471
contains                  0.779      0.591      0.733       3904       9280
film_performance          0.781      0.557      0.723        766       6142
founders                  0.739      0.366      0.614        380       5756
genre                     0.551      0.224      0.426        170       5546
has_sibling               0.793      0.222      0.524        499       5875
has_spouse                0.850      0.306      0.628        594       5970
is_a                      0.623      0.229      0.464        497       5873
nationality               0.500      0.146      0.337        301       5677
parents     

In [ ]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. We also ask that you report the best score your system got during development, just to help us understand how systems performed overall.

In [ ]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# My peak score was: MY_NUMBER
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass

# STOP COMMENT: Please do not remove this comment.

# check the best combinations of 4 featurizers:
    # directional_bag_of_words_featurizer, middle_bigram_pos_tag_featurizer synset_featurizer
    # and directional_bigram_bag_of_words_featurizer (new)

In [ ]:
def directional_bigram_bag_of_words_featurizer(kbt, corpus, feature_counter):
    """directional bigram BOW featurizer
    """
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        bigrams = get_bigrams(ex.middle)
        for bigram in bigrams:
            key = bigram + subject_object_suffix
            feature_counter[key] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        bigrams = get_bigrams(ex.middle)
        for bigram in bigrams:
            key = bigram + object_subject_suffix
            feature_counter[key] += 1
    return feature_counter


def get_bigrams(s, start_symbol= "<s>", end_symbol="</s>"):
    ws = s.strip().split(' ')
    ws = [start_symbol] + ws + [end_symbol]
    bigrams = [f'{ws[i-1]} {ws[i]}' for i in range(1, len(ws))]
    return bigrams

In [ ]:
def directional_middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for bigram_tag in get_tag_bigrams(ex.middle_POS):
            key = bigram_tag + subject_object_suffix
            feature_counter[key] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for bigram_tag in get_tag_bigrams(ex.middle_POS):
            key = bigram_tag + object_subject_suffix
            feature_counter[key] += 1

    return feature_counter

In [ ]:
_ = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[directional_bag_of_words_featurizer, middle_bigram_pos_tag_featurizer],  # synset_featurizer, directional_bigram_bag_of_words_featurizer
    verbose=True)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.833      0.397      0.683        340       5716
author                    0.819      0.650      0.779        509       5885
capital                   0.633      0.326      0.533         95       5471
contains                  0.800      0.687      0.775       3904       9280
film_performance          0.808      0.671      0.776        766       6142
founders                  0.745      0.432      0.651        380       5756
genre                     0.667      0.376      0.578        170       5546
has_sibling               0.845      0.285      0.606        499       5875
has_spouse                0.877      0.360      0.682        594       5970
is_a                      0.657      0.312      0.538        497       5873
nationality               0.624      0.276      0.498        301       5677
parents     

In [ ]:
_ = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[directional_bigram_bag_of_words_featurizer, middle_bigram_pos_tag_featurizer],  # synset_featurizer, directional_bigram_bag_of_words_featurizer
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.851      0.403      0.696        340       5716
author                    0.863      0.668      0.815        509       5885
capital                   0.592      0.305      0.498         95       5471
contains                  0.766      0.791      0.771       3904       9280
film_performance          0.867      0.704      0.828        766       6142
founders                  0.841      0.445      0.714        380       5756
genre                     0.778      0.412      0.660        170       5546
has_sibling               0.878      0.289      0.623        499       5875
has_spouse                0.895      0.372      0.698        594       5970
is_a                      0.696      0.322      0.565        497       5873
nationality               0.678      0.329      0.559        301       5677
parents     

In [ ]:
_ = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[directional_bigram_bag_of_words_featurizer, middle_bigram_pos_tag_featurizer,
                 synset_featurizer],  # synset_featurizer, directional_bigram_bag_of_words_featurizer
    verbose=True)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.808      0.371      0.654        340       5716
author                    0.840      0.672      0.800        509       5885
capital                   0.562      0.284      0.470         95       5471
contains                  0.754      0.787      0.761       3904       9280
film_performance          0.832      0.719      0.807        766       6142
founders                  0.769      0.455      0.676        380       5756
genre                     0.748      0.453      0.662        170       5546
has_sibling               0.815      0.283      0.592        499       5875
has_spouse                0.840      0.372      0.671        594       5970
is_a                      0.636      0.330      0.536        497       5873
nationality               0.618      0.339      0.531        301       5677
parents     

In [ ]:
best_result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[directional_bigram_bag_of_words_featurizer, directional_middle_bigram_pos_tag_featurizer],
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.827      0.409      0.687        340       5716
author                    0.887      0.662      0.830        509       5885
capital                   0.629      0.232      0.468         95       5471
contains                  0.792      0.757      0.785       3904       9280
film_performance          0.883      0.680      0.833        766       6142
founders                  0.858      0.397      0.696        380       5756
genre                     0.779      0.394      0.652        170       5546
has_sibling               0.878      0.244      0.578        499       5875
has_spouse                0.925      0.352      0.698        594       5970
is_a                      0.720      0.306      0.567        497       5873
nationality               0.700      0.302      0.554        301       5677
parents     

## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [ ]:
# Enter your bake-off assessment code in this cell.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE
    # rel_ext.bake_off_experiment(best_result, rel_ext_data_home)
#### DO NOT HAVE THE TEST FILE ###

In [ ]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported
# by the code above. Please enter only a number between
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
